### Homework Introduction. 
#### Author: Glen Lopez


### Q1. Running Elastic

In [1]:
# Run Elastic Search 8.17.6, and get the cluster information. What's the version.build_hash value?

In [2]:
# "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73"

In [3]:
# Get data

In [1]:
import requests
from tqdm.auto import tqdm
from elasticsearch import Elasticsearch
import tiktoken
from dotenv import load_dotenv
import ollama

In [3]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [4]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

### Q2. Indexing the data

In [5]:

es_client = Elasticsearch('http://127.0.0.1:9200')

In [6]:
es_client.info()

ObjectApiResponse({'name': '83285aca3a71', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'udwJG7RbRqydCWoWDkqmzg', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [7]:
index_setup = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

In [8]:
index_name = 'courses-questions-z'
es_client.indices.create(index=index_name, body=index_setup)

/var/folders/fb/rch1cgk167v8y_533l1hd97h0000gn/T/ipykernel_91307/487644483.py:2: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es_client.indices.create(index=index_name, body=index_setup)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'courses-questions-z'})

In [10]:
# adding data to elastic
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [15]:
## Which function do you use for adding your data to elastic?
# ans: index

### Q3. Searching

In [16]:
### Use only question and text fields and give question a boost of 4, and use "type": "best_fields"

In [11]:
query = 'How do execute a command on a Kubernetes pod?'

In [12]:
query_search = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}

In [13]:
res = es_client.search(index=index_name, body=query_search)

/var/folders/fb/rch1cgk167v8y_533l1hd97h0000gn/T/ipykernel_91307/2943586978.py:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res = es_client.search(index=index_name, body=query_search)


In [14]:
## What's the score for the top ranking result?

In [15]:
results_scores = []
for hit in res['hits']['hits']:
    results_scores.append(hit['_score'])

In [16]:
sorted(results_scores, reverse=True)

[32.70257, 31.96334, 31.96334, 30.263474, 30.263474]

In [17]:
results_scores

[32.70257, 31.96334, 31.96334, 30.263474, 30.263474]

### Q4. Filtering

In [18]:
question = 'How do copy a file to a Docker container?'

In [19]:
query_search = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": question,
                    "fields": ["question^4", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

In [21]:
index_name = 'courses-questions-z'

In [22]:
response = es_client.search(index=index_name, body=query_search)

/var/folders/fb/rch1cgk167v8y_533l1hd97h0000gn/T/ipykernel_91307/3438708907.py:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es_client.search(index=index_name, body=query_search)


In [23]:
response

ObjectApiResponse({'took': 43, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 342, 'relation': 'eq'}, 'max_score': 74.60149, 'hits': [{'_index': 'courses-questions-z', '_id': 'XE6BeJcBLPKq0K_TQvTa', '_score': 74.60149, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'courses-questions-z', '_id': 'e06BeJcBLPKq0K_TQ_Qg', '_score': 68.79335, '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\

In [24]:
results_sources = []
for hit in response['hits']['hits']:
    results_sources.append(hit['_source'])

In [25]:
results_sources

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

### Q5. Building a prompt

In [26]:
# Take the records returned from Elasticsearch in Q4 and use this template to build the context. 
# Separate context entries by two linebreaks (\n\n)
# question = 'How do copy a file to a Docker container?'

In [27]:
def build_context(search_results):
    context = ""
    for doc in search_results:
        context += f"Q: {doc['question']}\nA: {doc['text']}\n\n"
    return context.strip()

In [28]:
context = build_context(results_sources)

In [29]:
context

'Q: How do I debug a docker container?\nA: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nQ: How do I copy files from my local machine to docker container?\nA: You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani\n\nQ: How do I copy files from a different folder into docker container’s working directory?\nA: You can copy files from your local machine into a Docker container using the docker cp comman

In [30]:
def build_prompt(search_results, user_question):
    context = build_context(search_results)
    prompt_template = """
        You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
        Use only the facts from the CONTEXT when answering the QUESTION.
        
        QUESTION: {question}
        
        CONTEXT:
        {context}
        """.strip()
    return prompt_template.format(context=context, question=user_question)

In [31]:
prompt = build_prompt(results_sources, question)

In [32]:
len(prompt)

1494

### Q6. Tokens

In [33]:
# Let's calculate the number of tokens in our query:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [34]:
encodes = encoding.encode(prompt)

In [35]:
len(encoding.encode(prompt))

326

In [36]:
encodes[0]

63842

In [37]:
encoding.decode_single_token_bytes(63842)

b"You're"

### Bonus: generating the answer

In [38]:
def llm(prompt):
    response = ollama.chat(
        model='llama3',
        messages=[{'role': 'user', 'content': prompt}])
    return response['message']['content']

In [ ]:
answer = llm(prompt) # [W 2025-06-16 05:28:15.791 ServerApp] The websocket_ping_timeout (90000) cannot be longer than the websocket_ping_interval (30000).
    # Setting websocket_ping_timeout=30000